In [ ]:
import numpy as np

from modspectra.cube import EmissionCube
import astropy.units as u
from astropy.coordinates import SkyCoord

import matplotlib.pyplot as plt
%matplotlib notebook
import seaborn as sns
from matplotlib.colors import LogNorm
pal = sns.color_palette('colorblind', 10)

sns.set(color_codes=True)
sns.axes_style("white")
sns.set_style('whitegrid')

import copy
import scipy.io.idl
import matplotlib.gridspec as gridspec

In [ ]:
# Create Model
model_ha = EmissionCube.create_DK19(resolution = (64,64,64))

In [ ]:
sub_model = model_ha.spectral_slab(-110*u.km/u.s, -50*u.km/u.s)

In [ ]:
# Synthetic WHAM Observation helper functions
def wham_obs(cube, wham_pointing, moment = False, spectral_range = None, order = None):
    # select target coordinates
    lon = wham_pointing["GLON"]
    lat = wham_pointing["GLAT"]
    ds9_str = 'galactic; circle({0:.3}, {1:.4}, 0.5)'.format(lon, lat) 
    subcube = cube.subcube_from_ds9region(ds9_str)
    if moment:
        if spectral_range == None:
            moment = subcube.moment(order = order)
        else:
            moment = subcube.spectral_slab(spectral_range).moment(order = order)
        spectrum = np.nanmean(moment.value) * moment.unit
    else:
        spectrum = subcube.mean(axis = (1,2))
            
    return spectrum

def wham_observations(cube, wham_pointings, noise = 0.01, **kwargs):
    output_pointings = copy.deepcopy(wham_pointings)
    if 'moment' in kwargs:
        if 'order' in kwargs:
            if kwargs["order"] == 1:
                moment_values = np.empty(len(output_pointings)) << u.m/u.s
            else:
                moment_values = np.empty(len(output_pointings)) << u.R
    for ell, pointing in enumerate(output_pointings):
        spectrum = wham_obs(cube, pointing)
        moment_values[ell] = wham_obs(cube, pointing, **kwargs)
        output_pointings[ell]["DATA"] = spectrum.to(u.R/(u.km/u.s)).value
        output_pointings[ell]["VAR"] = (noise * np.abs(np.random.randn(len(pointing["VAR"]))))**2.
    return output_pointings, moment_values

In [ ]:
wham_baade = scipy.io.idl.readsav('baade_ha.sav', python_dict = True)

In [ ]:
model_wham_obs, model_wham_mom = wham_observations(model_ha, 
                                                   wham_baade["baade"], 
                                                   moment = True, 
                                                   order = 0)

In [ ]:
_, model_wham_vel = wham_observations(sub_model, 
                                                   wham_baade["baade"], 
                                                   moment = True, 
                                                   order = 1)

In [ ]:
lon = model_wham_obs["GLON"]
lat = model_wham_obs["GLAT"]
ha_int = np.log10(wham_baade["baade_mom"][:,0])
ha_vel = wham_baade["baade_mom"][:,1]
model_ha_int = np.log10(model_wham_mom.value)
model_ha_vel = model_wham_vel.to(u.km/u.s)

intensity_msk = ha_int > -1.
mean_velocity_msk = (ha_vel < -55.) & (ha_vel > -90.)

fitting_msk = intensity_msk & mean_velocity_msk
#Manual additons to mask
fitting_msk[20] = False # Manually Masked

In [ ]:
# Set points for highlighting
bpt_points = [1, 9, 12, 20]
nii_points = [2, 3, 4, 7, 8, 15]

In [ ]:
wham_baade["baade"]["GLON"][bpt_points]

In [ ]:
wham_baade["baade"].dtype.names

In [ ]:
wham_baade["baade"]["GLAT"][bpt_points]

In [ ]:
fig = plt.figure(figsize = (9,9))
ax = fig.add_subplot(111)

s =3300
vmin = -1.
vmax = 1


sc = ax.scatter(lon,lat,s = s, c = model_ha_int, vmin= vmin, vmax = vmax, cmap = 'Reds')
scv = ax.scatter(lon,lat,s = s/10, c = model_ha_vel, vmin= -110, vmax = -40, cmap = 'RdBu_r')

ax.scatter(lon[fitting_msk],lat[fitting_msk],facecolors='none', edgecolors='black', s = s, lw = 2)
    
ax.invert_xaxis()

cbar = plt.colorbar(sc)
cbar2 = plt.colorbar(scv, orientation = 'horizontal')
cbar2.set_label(r'Mean Velocity (km/s)', fontsize = 15)
cbar.set_label(r'Log H$\alpha$ Intensity (Log Rayleighs)', fontsize = 15)
cbar.ax.tick_params(labelsize = 15)
cbar2.ax.tick_params(labelsize = 15)

ax.set_xlabel('Galactic Longitude (deg)', fontsize = 15)
ax.set_ylabel('Galactic Latitude (deg)', fontsize = 15)
ax.tick_params(labelsize = 15)
ax.set_aspect('equal')

xlim_map = ax.get_xlim()
ylim_map = ax.get_ylim()

# Figure S2

In [ ]:
fig = plt.figure(figsize = (10,9))
gs = gridspec.GridSpec(14, 12, figure = fig, wspace = 0.02, hspace = 0.02)

vel = model_ha.spectral_axis.to(u.km/u.s).value

supax = fig.add_subplot(gs[:,:])
supax.grid(False)
supax.set_xlim(xlim_map)
supax.set_ylim(ylim_map)
supax.set_xlabel('Galactic Longitude (deg)', fontsize = 15)
supax.tick_params(labelsize = 15)
supax.set_ylabel('Galactic Latitude (deg)', fontsize = 15)


axes = []

axes.append(fig.add_subplot(gs[6:8,9:11]))
axes.append(fig.add_subplot(gs[6:8,7:9]))
axes.append(fig.add_subplot(gs[6:8,5:7]))
axes.append(fig.add_subplot(gs[6:8,3:5]))
axes.append(fig.add_subplot(gs[6:8,1:3]))

axes.append(fig.add_subplot(gs[4:6,0:2]))
axes.append(fig.add_subplot(gs[4:6,2:4]))
axes.append(fig.add_subplot(gs[4:6,4:6]))
axes.append(fig.add_subplot(gs[4:6,6:8]))
axes.append(fig.add_subplot(gs[4:6,8:10]))
axes.append(fig.add_subplot(gs[4:6,10:12]))

axes.append(fig.add_subplot(gs[2:4,9:11]))
axes.append(fig.add_subplot(gs[2:4,7:9]))
axes.append(fig.add_subplot(gs[2:4,5:7]))
axes.append(fig.add_subplot(gs[2:4,3:5]))
axes.append(fig.add_subplot(gs[2:4,1:3]))

axes.append(fig.add_subplot(gs[0:2,0:2]))
axes.append(fig.add_subplot(gs[0:2,2:4]))
axes.append(fig.add_subplot(gs[0:2,4:6]))
axes.append(fig.add_subplot(gs[0:2,6:8]))
axes.append(fig.add_subplot(gs[0:2,8:10]))
axes.append(fig.add_subplot(gs[0:2,10:12]))

axes.append(fig.add_subplot(gs[12:14,10:12]))
axes.append(fig.add_subplot(gs[12:14,8:10]))
axes.append(fig.add_subplot(gs[12:14,6:8]))
axes.append(fig.add_subplot(gs[12:14,4:6]))
axes.append(fig.add_subplot(gs[12:14,2:4]))
axes.append(fig.add_subplot(gs[12:14,0:2]))

axes.append(fig.add_subplot(gs[10:12,1:3]))
axes.append(fig.add_subplot(gs[10:12,3:5]))
axes.append(fig.add_subplot(gs[10:12,5:7]))
axes.append(fig.add_subplot(gs[10:12,7:9]))
axes.append(fig.add_subplot(gs[10:12,9:11]))

axes.append(fig.add_subplot(gs[8:10,10:12]))
axes.append(fig.add_subplot(gs[8:10,8:10]))
axes.append(fig.add_subplot(gs[8:10,6:8]))
axes.append(fig.add_subplot(gs[8:10,4:6]))
axes.append(fig.add_subplot(gs[8:10,2:4]))
axes.append(fig.add_subplot(gs[8:10,0:2]))

for ell, ax in enumerate(axes):
    x = wham_baade["baade"][ell]["VEL"]
    y = wham_baade["baade"][ell]["DATA"]
    vel_to_plot = (x <= -20) & (x >= -150)
    if ell == 0:
        ax.plot(x[vel_to_plot], y[vel_to_plot], "-", color = pal[0], label = "WHAM")
    else:
        ax.plot(x[vel_to_plot], y[vel_to_plot], "-", color = pal[0])
    xlim = ax.get_xlim()
    ylim = ax.set_ylim()
    if ell == 0:
        ax.plot(vel, model_wham_obs[ell]["DATA"], "--", color = pal[1], lw = 3, label = "Model")
    else:
        ax.plot(vel, model_wham_obs[ell]["DATA"], "--", color = pal[1], lw = 3)
        
    ax.fill_betweenx([-.01, .04], [-40, -40], [0,0], 
                         color = "k", alpha = 0.1, zorder = 0)
    ax.set_xlim(xlim)
    if fitting_msk[ell]:
        ax.spines['bottom'].set_linewidth(6)
        ax.spines['bottom'].set_color("k")
        ax.spines['top'].set_linewidth(6)
        ax.spines['top'].set_color("k")
        ax.spines['right'].set_linewidth(4)
        ax.spines['right'].set_color("k")
        ax.spines['left'].set_linewidth(4)
        ax.spines['left'].set_color("k")
        ax.fill_betweenx([-.01, .04], [-45, -45], [-110, -110], 
                         color = pal[2], alpha = 0.2, zorder = 0)
    if ell in bpt_points:
        ax.text(-105, 0.024, "*", fontsize = 35, color = pal[3], weight='bold',
                horizontalalignment='center', 
                verticalalignment='center')
    if ell in nii_points:
        ax.text(-105, 0.028, "#", fontsize = 20, color = pal[4], weight='bold',
                horizontalalignment='center', 
                verticalalignment='center')
    ax.set_ylim([-0.01,0.04])
    ax.grid(False)
    ax.set_xticks([])
    ax.set_yticks([])

fig.text(0.39, 0.89, "Observed (WHAM)", ha="center", va="bottom", size=16,color=pal[0])
fig.text(0.5, 0.89, "&", ha="center", va="bottom", size=16)
fig.text(0.56,0.89,"Modeled", ha="center", va="bottom", size=16,color=pal[1])
fig.text(0.645,0.89,"Spectra", ha="center", va="bottom", size=16,color="k")

plt.tight_layout()

# plt.savefig("FigureS3.png", dpi = 300, transparent = True)
# plt.savefig("FigureS3.svg", transparent = True)